<a href="https://colab.research.google.com/github/serdar-bayramov/big-data-challenge/blob/main/notebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = "spark-3.2.0"
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_6

In [2]:
import pandas as pd
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 55.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=088a0d22b4a6da204457c9d625a7ec95cdee02e2df5f4ff8cb7573ad6e1b3054
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("big-data-challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Read mobile electronics reviews from amazon reviews

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
mobile_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
mobile_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [6]:
mobile_df.count()

104975

Create Review_ID_df table 

In [7]:
from pyspark.sql.functions import to_date
review_ID_df = mobile_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_ID_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8MEA6IGAHO0B|   20422322|B00MC4CED8|     217304173| 2015-08-31|
|R31LOQ8JGLPRLK|   40835037|B00OQMFG1Q|     137313254| 2015-08-31|
|R2Y0MM9YE6OP3P|   51469641|B00QERR5CY|      82850235| 2015-08-31|
| RRB9C05HDOD4O|    4332923|B00QUFTPV4|     221169481| 2015-08-31|
|R26I2RI1GFV8QG|   44855305|B0067XVNTG|     563475445| 2015-08-31|
| RY8DDL22YG4R5|    7846966|B00KA6CCVY|     713636156| 2015-08-31|
|R2AT2426ZHFUHH|   21299354|B00MJCDPM2|     754171872| 2015-08-31|
|R3RRXU2R23NMQ9|   28902968|B00ET5AWBY|     508373185| 2015-08-31|
|R250PR8VJUZ62F|    2074438|B00Z9K379C|     759387370| 2015-08-31|
| RBEMQ29WJBHYG|   48701722|B00YO3UYXW|     913911909| 2015-08-31|
|R19VVIUT4BZCMT|    2990247|B011W26BU2|     938399424| 2015-08-31|
|R1DT8JJUQHUKUL|   52946117|B00H8PI78C|     661515294| 2015-08

Create customer_df table

In [8]:
customer_df = mobile_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|     176073|             1|
|   20992492|             1|
|    1397686|             1|
|   50371142|             1|
|     235096|             1|
|   12320806|             1|
|   19707203|             1|
|   15207172|             1|
|     538703|             1|
|   11223135|             1|
|   31447151|             1|
|   35961247|             1|
|   17835519|             1|
|   15404885|             1|
|   10182610|             1|
|   52399092|             1|
|   17133428|             1|
|   27870423|             1|
|   11574168|             1|
|   30685607|             1|
+-----------+--------------+
only showing top 20 rows



Create products_df table


In [9]:
products_df = mobile_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01075GRAO|ANYSECU QYT KT890...|
|B00QVXNV60|BRG AH2000G-ANR A...|
|B004ECO5VA|Holux Mini Smart ...|
|B00LGNKLXG|Aweek® Car Antenn...|
|B00M1Y87I0|      sentey ls 4063|
|B002ABX29I|Micca Slim Portab...|
|B00XXA2T2G|Centurion GT1 Car...|
|B00Q5A5PD6|HYUNDAI HCR-580 V...|
|B00VFEK8IY|Z ZTDM Mini Stere...|
|B00L85M8VU|Settonbrothers Ul...|
|B00SUXD5GU|[NEW RELEASE] JEB...|
|B00KJ7MVYW|Fincibo (TM) Appl...|
|B00A6YQQS0|Minisuit Aero Fol...|
|B009S99IYI|PIO TSG1644R 6.5 ...|
|B006PAZCXY|eForCity® Anti-Gl...|
|B00OXD61OE|SHE'O® 2x Cool No...|
|B000IBMI1K|RCA 200-Watt Mono...|
|B00UN6955M|Aisster(TM) Lot 1...|
|B00V684NCQ|LSD 8GB Flash Dri...|
|B00A609WOO|eForCity Hybrid S...|
+----------+--------------------+
only showing top 20 rows



Create reviews_df table

In [10]:
reviews_df = mobile_df.select(["review_id", "review_headline", "review_body"])
reviews_df.show()

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
| R8MEA6IGAHO0B|         Very Happy!|As advertised. Ev...|
|R31LOQ8JGLPRLK|           five star|          it's great|
|R2Y0MM9YE6OP3P|        great cables|These work great ...|
| RRB9C05HDOD4O|Work very well bu...|Work very well bu...|
|R26I2RI1GFV8QG|Cameras has batte...|Be careful with t...|
| RY8DDL22YG4R5|It appears to be ...|It appears to be ...|
|R2AT2426ZHFUHH|Didn't love the f...|First one arrived...|
|R3RRXU2R23NMQ9|          Five Stars|Worked great for ...|
|R250PR8VJUZ62F|          Four Stars|this is a very go...|
| RBEMQ29WJBHYG|  I LOVE my recorder|I LOVE my recorde...|
|R19VVIUT4BZCMT|          Five Stars|Great sound great...|
|R1DT8JJUQHUKUL|More of a beeper ...|If what you want ...|
| RLIL6S9OGM3YA|          Five Stars|             Love it|
|R34EZZ68VYPHO0|            One Star|               Suck

Create vine_df table

In [11]:
vine_df = mobile_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8MEA6IGAHO0B|          5|            0|          0|   N|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|
| RRB9C05HDOD4O|          4|            0|          0|   N|
|R26I2RI1GFV8QG|          2|            0|          0|   N|
| RY8DDL22YG4R5|          3|            0|          1|   N|
|R2AT2426ZHFUHH|          3|            0|          1|   N|
|R3RRXU2R23NMQ9|          5|            0|          0|   N|
|R250PR8VJUZ62F|          4|            0|          2|   N|
| RBEMQ29WJBHYG|          5|          164|        168|   N|
|R19VVIUT4BZCMT|          5|            2|          2|   N|
|R1DT8JJUQHUKUL|          3|            0|          1|   N|
| RLIL6S9OGM3YA|          5|            0|          0|   N|
|R34EZZ68VYPHO0|          1|            

Connect to Database

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<instance>"
config = {"user":<user>, 
          "password": <password>, 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_ID_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)